# Test of GAT
- use DGL
- predict `graphs`
- test: 0~99
- validation: 100~199
- train: 200~999
- larger lr with scheduler
- try the sklearn report

In [1]:
import os
import dgl
import json
import torch
import torch as th
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import classification_report


- check the GPU and assign the GPU by the best memory usage

In [2]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:1")
print(device)


cuda:0


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]

        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)

        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

        return g, th.tensor(data["label"]).to(self.device)


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [5]:
datasets = ['train', 'valid', 'test']
# datasets = ['test']
dataset_data = {}

for dataset_name in tqdm(datasets):
#     file_path = f"../../data_processing/dgl/data/test_graph/repeated_{dataset_name}.jsonl"
    file_path = f"../../data_processing/dgl/data_new/training_data/exp_2/transH_100/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/training_data/exp_2/transH_100/train.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/training_data/exp_2/transH_100/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/training_data/exp_2/transH_100/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

# dataloaders = create_dataloaders(4)
dataloaders = create_dataloaders(16)


if (len(dataloaders['test'].dataset) + len(dataloaders['valid'].dataset) + len(dataloaders['train'].dataset)) % 166 != 0: print("Error data!!")
else: print("OK!")


OK!


- Turn the print message to a log file

In [7]:
# print(dataloaders['test'][5])
sample = dataset_data['train'][5000]
print(sample)

print(len(dataloaders['test'].dataset))
print(len(dataloaders['valid'].dataset))
print(len(dataloaders['train'].dataset))
print(len(dataloaders['test'].dataset) + len(dataloaders['valid'].dataset) + len(dataloaders['train'].dataset))

(Graph(num_nodes=10, num_edges=19,
      ndata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32)}
      edata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32)}), tensor(6, device='cuda:0'))
16600
16600
132800
166000


In [8]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GAT_transH_100.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/0901_19:32_GAT_transH_100.log


### Model

In [9]:
class GAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, dropout_prob=0.25):
        super(GAT, self).__init__()
        
        # do not check the zero in_degree since we have all the complete graph
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads=num_heads, activation=F.relu, allow_zero_in_degree=True)
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, num_heads=num_heads, allow_zero_in_degree=True)
        
        # Adding Batch Normalization after each GAT layer
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim * num_heads)
        self.batchnorm2 = nn.BatchNorm1d(out_dim)
        
        # Adding Dropout for regularization
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, g, h):
        # Apply GAT layers
        h = self.layer1(g, h)
        h = h.view(h.shape[0], -1)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h).squeeze(1)
        
        # Store the output as a new node feature
        g.ndata['h_out'] = h

        # Use mean pooling to aggregate this new node feature
        h_agg = dgl.mean_nodes(g, feat='h_out')
        return h_agg

    

- Model Forward  

In [10]:
def model_fn(data, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
    batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
    logits = logits.mean(dim=1)
    
    loss = criterion(logits, labels)

    # Get the class id with the highest probability
    preds = logits.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [11]:
seed = 8787
same_seeds(seed)

model = GAT(in_dim=100, hidden_dim=16, out_dim=168, num_heads=8)
torch.save(model.state_dict(), 'model1_initial/initial_weight.pth')

In [12]:
model.layer1.fc.weight

Parameter containing:
tensor([[-0.0372,  0.0397,  0.0935,  ..., -0.2031, -0.0468, -0.0559],
        [-0.2012, -0.0840,  0.1344,  ...,  0.1873, -0.0882,  0.1701],
        [ 0.0989,  0.1005, -0.2093,  ..., -0.2812, -0.2957, -0.0144],
        ...,
        [-0.1528,  0.2374,  0.1282,  ..., -0.0509,  0.0520, -0.0736],
        [-0.0805, -0.1986, -0.1437,  ...,  0.1331,  0.0198,  0.0471],
        [ 0.0363,  0.0976, -0.1171,  ...,  0.0343, -0.0835,  0.2840]],
       requires_grad=True)

- Check if model really load the model_dict

In [13]:
model = GAT(in_dim=100, hidden_dim=16, out_dim=168, num_heads=8)
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))
model.layer1.fc.weight

Parameter containing:
tensor([[-0.0372,  0.0397,  0.0935,  ..., -0.2031, -0.0468, -0.0559],
        [-0.2012, -0.0840,  0.1344,  ...,  0.1873, -0.0882,  0.1701],
        [ 0.0989,  0.1005, -0.2093,  ..., -0.2812, -0.2957, -0.0144],
        ...,
        [-0.1528,  0.2374,  0.1282,  ..., -0.0509,  0.0520, -0.0736],
        [-0.0805, -0.1986, -0.1437,  ...,  0.1331,  0.0198,  0.0471],
        [ 0.0363,  0.0976, -0.1171,  ...,  0.0343, -0.0835,  0.2840]],
       requires_grad=True)

### test of valid and test part is ``graph``

- Batch size = 4
- use large lr and scheduler

In [14]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = GAT(in_dim=100, hidden_dim=16, out_dim=168, num_heads=8)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))
best_model_path = "../checkpoint_GAT/best_model_GAT_transH_100.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=5e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=18, num_training_steps=total_steps)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)


criterion = nn.CrossEntropyLoss()
total_steps = 25

# save the best model
best_val_loss = float('inf')
patience = 5  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

        
#     scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for data in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)

        
#         print(best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/25 [00:00<?, ?it/s]

Training:   0%|          | 0/8300 [00:00<?, ?it/s]

09/01/2023, 19:34:47# labels of 5000: tensor([ 67, 164, 109,  93,  40,  15,  66,  42,  37, 150, 120,  44,  96,  37,
        122,  33], device='cuda:0') torch.Size([16])
09/01/2023, 19:34:47# predicted of 5000: tensor([149, 132, 161, 132,  99,  52, 132, 132, 132, 132,  99, 132,  99, 132,
        165, 132], device='cuda:0') torch.Size([16])
09/01/2023, 19:36:35# total batches: 8300
09/01/2023, 19:36:35# Epoch 0 | Train Loss: 4.6242 | Train Accuracy: 0.0532


Validation:   0%|          | 0/1038 [00:00<?, ?it/s]

09/01/2023, 19:36:35# labels of Validation: tensor([140,  63, 123,  33,  63, 126,  14, 142, 131, 118,  57,   2,  40,   1,
        101,  64], device='cuda:0') torch.Size([16])
09/01/2023, 19:36:35# predicted of Validation: tensor([128,  28, 123,  28,  28, 120,  28,  28, 132, 118,  28,  28, 106,  28,
        132, 132], device='cuda:0') torch.Size([16])
09/01/2023, 19:36:35# labels of 0: tensor([140,  63, 123,  33,  63, 126,  14, 142, 131, 118,  57,   2,  40,   1,
        101,  64], device='cuda:0') torch.Size([16])
09/01/2023, 19:36:35# predicted of 0: tensor([128,  28, 123,  28,  28, 120,  28,  28, 132, 118,  28,  28, 106,  28,
        132, 132], device='cuda:0') torch.Size([16])
09/01/2023, 19:36:57# labels of Validation: tensor([ 79, 124,  70, 138, 154,  24,  60,  43, 106,  29, 138,  56, 112,  19,
        143,  46], device='cuda:0') torch.Size([16])
09/01/2023, 19:36:57# predicted of Validation: tensor([ 32,  28,  70, 138, 154,  28,  28,  28, 106,  29, 138, 149,  28,  28,
         28,

...

09/01/2023, 21:30:39# labels of Validation: tensor([ 95, 150, 101,  35, 117,  88,  79,  73,  28, 109,  52, 122,  27,  12,
         35,  75], device='cuda:0') torch.Size([16])
09/01/2023, 21:30:39# predicted of Validation: tensor([ 95, 162, 155,  35, 117, 155,  79,  73,   6, 162,  52, 122,  27, 162,
         35, 162], device='cuda:0') torch.Size([16])
09/01/2023, 21:30:39# labels of 0: tensor([ 95, 150, 101,  35, 117,  88,  79,  73,  28, 109,  52, 122,  27,  12,
         35,  75], device='cuda:0') torch.Size([16])
09/01/2023, 21:30:39# predicted of 0: tensor([ 95, 162, 155,  35, 117, 155,  79,  73,   6, 162,  52, 122,  27, 162,
         35, 162], device='cuda:0') torch.Size([16])
09/01/2023, 21:31:03# labels of Validation: tensor([ 92, 149, 152,   4,  30,  35,  12,   7,  14,   6, 143,  19, 129,  72,
         31,  31], device='cuda:0') torch.Size([16])
09/01/2023, 21:31:03# predicted of Validation: tensor([162, 149, 162, 162, 162,  35, 162,   7, 162,   6, 162, 162, 155,  29,
        162,

Training:   0%|          | 0/8300 [00:00<?, ?it/s]

09/01/2023, 21:33:54# labels of 5000: tensor([ 25,  92,  96, 152, 157, 138, 134,  15,  92,  25,  90,  34,  16,  82,
        100,  61], device='cuda:0') torch.Size([16])
09/01/2023, 21:33:54# predicted of 5000: tensor([ 25, 124,  96, 104,  12, 138, 134,  15, 125,  25, 155,  19,  16,  82,
         59,  61], device='cuda:0') torch.Size([16])
09/01/2023, 21:35:48# total batches: 8300
09/01/2023, 21:35:48# Epoch 24 | Train Loss: 2.7866 | Train Accuracy: 0.3733


Validation:   0%|          | 0/1038 [00:00<?, ?it/s]

09/01/2023, 21:35:48# labels of Validation: tensor([153, 113,  74,  74,  39, 153,   5,  61, 162,  28,   4, 128,  71, 124,
        100,  85], device='cuda:0') torch.Size([16])
09/01/2023, 21:35:48# predicted of Validation: tensor([ 71, 108,  71,  71,  39,  71,   5,  61,  71, 132,  71, 128,  71,  71,
        108,  85], device='cuda:0') torch.Size([16])
09/01/2023, 21:35:48# labels of 0: tensor([153, 113,  74,  74,  39, 153,   5,  61, 162,  28,   4, 128,  71, 124,
        100,  85], device='cuda:0') torch.Size([16])
09/01/2023, 21:35:48# predicted of 0: tensor([ 71, 108,  71,  71,  39,  71,   5,  61,  71, 132,  71, 128,  71,  71,
        108,  85], device='cuda:0') torch.Size([16])
09/01/2023, 21:36:12# labels of Validation: tensor([ 14,  10, 165,  35, 152,  36,  48, 101, 151,  45,  98, 124,   0,  68,
        121, 106], device='cuda:0') torch.Size([16])
09/01/2023, 21:36:12# predicted of Validation: tensor([ 71,  71, 165,  35,  71,  71,  71, 132,  71,  45,  71,  71,   0, 108,
         71,

### Testing Part

In [15]:
# load the pretrained model
pretrained_model_path = '../checkpoint_GAT/best_model_GAT_transH_100.pt'
model.load_state_dict(torch.load(pretrained_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for data in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):

        loss, accuracy, predicted = model_fn(data, model, criterion, device, count, which_type='test')
        labels = data[1].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transH_100-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transH_100-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

Testing:   0%|          | 0/1038 [00:00<?, ?it/s]

09/01/2023, 21:36:13# labels of Test: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:0') torch.Size([16])
09/01/2023, 21:36:13# predicted of Test: tensor([71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71],
       device='cuda:0') torch.Size([16])
09/01/2023, 21:36:13# labels of 0: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:0') torch.Size([16])
09/01/2023, 21:36:13# predicted of 0: tensor([71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71],
       device='cuda:0') torch.Size([16])
09/01/2023, 21:36:13# labels: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:0') torch.Size([16])
09/01/2023, 21:36:13# predicted: tensor([71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71],
       device='cuda:0') torch.Size([16])
09/01/2023, 21:36:36# labels of Test: tensor([162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/01/2023, 21:36:47# report path: classification_report/classification_report-transH_100-0.xlsx
09/01/2023, 21:36:47# label path: classification_report/mapped_true_predicted_labels-transH_100-0.xlsx


/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/01/2023, 21:36:48# mapped_report:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00       100
    T1003.001_35d92515122effdd73801c6ac3021da7       0.99      1.00      1.00       100
    T1003.002_5a484b65c247675e3b7ada4ba648d376       0.00      0.00      0.00       100
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00       100
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.00      0.00      0.00       100
    T1003.003_f049b89533298c2d6cd37a940248b219       0.00      0.00      0.00       100
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00       100
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00       100
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.00      0.00      0.00       100
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.00      0.00      0.00     

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
